In [1]:
import pandas as pd
import time
import numpy as np
from collections import defaultdict
from heavy_tail_observations import BothSideWeibullNoise, BothSideParetoNoise, BothSideFrechetNoise
from heavy_tail_lin_bandit import MENU, TOFU, SupHvyLinBandit

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv("/home/rai/다운로드/train_triplets.txt", sep="\t", names=['user', 'song', 'number of play'])
data = data[:100000]
data.head()

,user,song,number of play
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1


In [3]:
user = np.unique(data['user'].to_numpy())
len(user)

2042

In [ ]:
from pyechonest import catalog

In [3]:
song = np.unique(data['song'].to_numpy())
len(song)

384546

In [5]:
dt = data.to_numpy()
data_dict = defaultdict(list)
for i in dt:
    data_dict[i[0]].append({i[1]: i[2]})

In [6]:
df = {'user': user}
df = pd.DataFrame(df)
df.head()

,user
0,0007c0e74728ca9ef0fe4eb7f75732e8026a278b
1,000ebc858861aca26bac9b49f650ed424cf882fc
2,00342a0cdf56a45465f09a39040a5bc25b7d0046
3,0039bd8483d578997718cdc0bf6c7c88b679f488
4,00498f4bab2bfeb17680113c7d9525ad5b0ad401


In [7]:
df2 = pd.DataFrame(columns=song)
df2.head()

,SOAAAGQ12A8C1420C8,SOAABMP12A6D4F7633,SOAABVA12AC3DF673F,SOAACPJ12A81C21360,SOAACTC12AB0186A20,SOAADCB12A81C22AFA,SOAAEJI12AB0188AB5,SOAAEKX12A6D4F7E4E,SOAAFAC12A67ADF7EB,SOAAFBZ12AB01842D7,...,SOZZWTD12AF72A4FA1,SOZZWZV12A67AE140F,SOZZXAO12A58A7D379,SOZZXOX12A8AE47ED6,SOZZXZP12AB0180443,SOZZYAO12A6701FF36,SOZZYDA12AB01824FB,SOZZYMH12AB0180A51,SOZZZBY12A8C139058,SOZZZFB12A8AE45CDC


In [8]:
df = pd.concat([df, df2], axis=1)
df = df.fillna(0)
df.head()

,user,SOAAAGQ12A8C1420C8,SOAABMP12A6D4F7633,SOAABVA12AC3DF673F,SOAACPJ12A81C21360,SOAACTC12AB0186A20,SOAADCB12A81C22AFA,SOAAEJI12AB0188AB5,SOAAEKX12A6D4F7E4E,SOAAFAC12A67ADF7EB,...,SOZZWTD12AF72A4FA1,SOZZWZV12A67AE140F,SOZZXAO12A58A7D379,SOZZXOX12A8AE47ED6,SOZZXZP12AB0180443,SOZZYAO12A6701FF36,SOZZYDA12AB01824FB,SOZZYMH12AB0180A51,SOZZZBY12A8C139058,SOZZZFB12A8AE45CDC
0,0007c0e74728ca9ef0fe4eb7f75732e8026a278b,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,000ebc858861aca26bac9b49f650ed424cf882fc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,00342a0cdf56a45465f09a39040a5bc25b7d0046,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0039bd8483d578997718cdc0bf6c7c88b679f488,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,00498f4bab2bfeb17680113c7d9525ad5b0ad401,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
